In [157]:
import requests
from bs4 import BeautifulSoup
import pycantonese
import re

# Data Soruce: https://cantonese.ca/ 

corpus = pycantonese.hkcancor()

url = "https://cantonese.ca/"
main_html = requests.get(url).text
main_soup = BeautifulSoup(main_html, "lxml")


In [158]:
class Group:
    def __init__(self, name, link):
        self.name = name
        self.link = link
        self.html = requests.get(url+self.link).text
        self.soup = BeautifulSoup(self.html, "lxml")
        self.word_dict = {}
    # add a to dic method https://stackoverflow.com/questions/34997174/how-to-convert-list-of-model-objects-to-pandas-dataframe/41762270 

In [159]:
groups = []

for link in main_soup.find_all('a'):
    if re.search(r"[\u4e00-\u9fff]", link.decode_contents()) or re.search(r"[^a-zA-Z0-9\s]", link.decode_contents()):
        continue
    groups.append(Group(link.decode_contents(), link.get("href")))

del groups[0:4]
del groups[-2:]

In [173]:
del_gps = ["Comparisons and similes", "Romanization conversion", "Time", "Zodiac", "Idioms and Metapthors", "Links", "Smoking", "Measure words", "Prefixes and suffixes", "Slang", "Swearing"]


for gp in groups:

    # remove unwanted groups
    if gp.name in del_gps:
        groups.remove(gp)
        continue

    gp.word_dict["English"] = []
    gp.word_dict["Cantonese"] = []

    for l in gp.soup.find_all('tr'):
        td = l.findChildren("td")[:-1]

        
        # for Colours --> special order
        if gp.name == "Colours":
            if len(td) < 2:
                continue
            if td[3].get_text() == "white":
                gp.word_dict["English"].append(td[3].get_text())
                gp.word_dict["Cantonese"].append(td[4].get_text())
            else:
                gp.word_dict["English"].append(td[2].get_text())
                gp.word_dict["Cantonese"].append(td[3].get_text())
        else:
            # wrong length or no eng and ch
            if len(td) < 2 or (len(td[0]) == 0 and len(td[1]) == 0):
                continue
            
            gp.word_dict["English"].append(td[0].get_text())
            if not(re.search(r"[\u4e00-\u9fff]", td[1].get_text())):
                gp.word_dict["Cantonese"].append('')
            else: 
                gp.word_dict["Cantonese"].append(td[1].get_text())

"""
for a in groups:
    print(a.name)
    print(a.word_dict)
    print(len(a.word_dict["Eng"]), len(a.word_dict["Ch"]))
    print()
"""

'\nfor a in groups:\n    print(a.name)\n    print(a.word_dict)\n    print(len(a.word_dict["Eng"]), len(a.word_dict["Ch"]))\n    print()\n'

In [174]:
def fix(string):
    temp = ''.join([c for c in string if not(str(c.encode('utf-8')) == "b' '")])
    ''.join(temp)

    if re.match(r'.+\n.+', temp):
        temp = temp.replace('\n', r'/')
    if re.match(r'.+/$', temp) or re.match(r'.+\n$', temp) :
        temp = temp[:-1]
    return temp

for gp in groups:
    if len(gp.word_dict['English']) == 0 and len(gp.word_dict['Cantonese']) == 0:
        groups.remove(gp)

    if len(gp.word_dict['English']) < 2:
        continue
    
    del_idx = []
    for i in range(len(gp.word_dict['English'])-1):
        if gp.word_dict['English'][i] == '' and gp.word_dict['Cantonese'][i] == '':
            del_idx.append(i)
            continue
        
        #gp.word_dict['English'][i] = fix(gp.word_dict['English'][i])
        gp.word_dict['Cantonese'][i] = fix(gp.word_dict['Cantonese'][i])
    
    if len(del_idx) > 0:
        for i in del_idx:
            del gp.word_dict['English'][i]
            del gp.word_dict['Cantonese'][i]


In [177]:
import pandas as pd


df = pd.DataFrame.from_dict(groups[2].word_dict)
df
df


,English,Cantonese
0,zero,零
1,one,一
2,two,二兩
3,three,三
4,four,四
5,five,五
6,six,六
7,seven,七
8,eight,八
9,nine,九


In [6]:
from translator import translate

translate('Good Morning', 'en', ['yue', 'zh-Hans'])

['早晨', '早上好']

In [8]:
for w in words:
    t = pycantonese.parse_text(w['Ch'])
    #if t.length > 0:
    print(t.head())
    print(t.jyutping())

NameError: name 'words' is not defined